## 字典格式

In [76]:
## 字典格式
article = {
    'comp':'Gossip',
    'category':'問卦',
    'url':'',
    'author':'',
    'title':'',
    'date':'',
    'push_nums':'',
    'main_content':'',
    'push_content':[  #推文會有多個，所以用list裝
        {
            'tag':'',
            'userid':'',
            'datetime':'',
            'content':'',
        }
    ],
    'push_count':{  #算數量的只會有一個，用字典裝就好
            'all':'',
            'good':'',
            'bad':'',
            'neutral':'',
            'my_push_nums':'good-bad',
    }
    
}

## 抓日期

In [4]:
import datetime
today = datetime.datetime.now()
yestoday = today + datetime.timedelta(days = -1)
yestoday = yestoday.strftime('%Y%m%d')
fileName = yestoday
# print fileName

20160925


## Step1 讀取Url File 爬取內文 

In [2]:
#coding=utf-8 
import requests
from bs4 import BeautifulSoup
import time
import json

#通過18禁頁面
playload = {
    'from':'/bbs/Gossiping/index.html',
    'yes':'yes'
}
rs = requests.session() #設定一個session
res1 = rs.post('https://www.ptt.cc/ask/over18',verify=False,data = playload) #透過session記錄通過18禁頁面

#基本設定
a = 0 #限制跑幾圈
errorCount = 0
PttName="Gossiping"
all_json_str = [] #存所有資訊的 json

#開啟 url檔案
with open('C:/Users/ytchen/Desktop/ptt/{}.txt'.format(fileName),'r') as f:
    
    for line in f.readlines():
        tempList = line.split(',')
        
        try:
            #Get Information
            url = 'http'+tempList[0].split('http')[1] #截取下來的網址，http前面會有奇怪的東西，要刪掉
            category = tempList[1]
            push_nums = tempList[2]
            title = tempList[3]
            date = tempList[4]
            author = tempList[5]      
    #         print line
    #         print url,category,push_nums,title,date,auth

            #開始爬取文章
            res = rs.get(url,verify=False)
            soup = BeautifulSoup(res.text)

            #mydate
            mydate = soup.select('.article-meta-value')[3].text

            #content
            content = soup.find(id="main-content").text
            target_content=u'※ 發信站: 批踢踢實業坊(ptt.cc),'
            content = content.split(target_content)
            content = content[0].split(mydate)
            main_content = content[1].replace('\n', ' ').replace('\t', ' ')

            # message
            all_push_content = []
            num , good , bad , neutral = 0,0,0,0

            for tag in soup.select('div.push'):
                num += 1
                push_tag = tag.find("span", {'class': 'push-tag'}).text
                push_userid = tag.find("span", {'class': 'push-userid'}).text      
                push_content = tag.find("span", {'class': 'push-content'}).text.replace('\n', '').replace('\t', '').strip()
                push_content = push_content[1:].strip()
    #             print "push_content:",push_content
                push_ipdatetime = tag.find("span", {'class': 'push-ipdatetime'}).text.replace('\n', '').replace('\t', '').strip()

                message = {
                    'tag':push_tag,
                    'userid':push_userid,
                    'content':push_content,
                    'datetime':push_ipdatetime,
                }

                #存取所有推文
                all_push_content.append(message)

                #計算推文數量
                if push_tag == u'推 ':
                    good += 1
                elif push_tag == u'噓 ':
                    bad += 1
                else:
                    neutral += 1

            push_count = {
                'all':num,
                'good':good,
                'bad':bad,
                'neutral':neutral,
                'my_push_nums':good-bad,
            }

    #         print all_push_content
    #         print push_count

            #存取每一篇資訊
            ptt_news = {
                'comp':PttName,
                'category':category,
                'url':url,
                'author':author,
                'title':title,
                'date':date,
                'push_nums':push_nums,
                'main_content':main_content,
                'push_content':all_push_content,
                'push_count':push_count,
            }
            all_json_str.append(ptt_news)
            time.sleep(2)

            #限制迴圈次數
    #         a += 1
    #         if a >= 1 :
    #             break
        except:
            errorCount += 1
            continue

#檔案儲存
with open('C:/Users/ytchen/Desktop/ptt/json/{}.json'.format(fileName),'w') as f:
    b = json.dumps(all_json_str) #再把 list 字典轉換成 json 並包起來
    f.write(b)

#log記錄
logString = ''
fileName_log = fileName+'/'+'errorCount:'+str(errorCount)+'\n'
#讀取
with open('C:/Users/ytchen/Desktop/ptt/log_content.txt','r') as f1:
    for logLine in f1.readlines():
        logString += logLine
#重新寫入
with open('C:/Users/ytchen/Desktop/ptt/log_content.txt','w') as f2:
    logString += fileName_log
    f2.write(logString.encode('utf-8'))

C:\Anaconda2\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


## 存入MongoDB

In [1]:
import pymongo
from pymongo import MongoClient
import json
import datetime

# #抓日期
# today = datetime.datetime.now()
# yestoday = today + datetime.timedelta(days = -2)
# yestoday = yestoday.strftime('%Y%m%d')
# fileName = yestoday

f = open('C:/Users/ytchen/Desktop/ptt/json/{}.json'.format('20161006'),'r')
i = f.read()
json = json.loads(i)
    
client = MongoClient() 
database = client["test"]
collection = database["ptt"]
collection.insert_many(
    json
)
client.close()
f.close()

## 練習用

In [ ]:
# #coding=utf-8 
# import requests
# from bs4 import BeautifulSoup

# playload = {
#     'from':'/bbs/Gossiping/index.html',
#     'yes':'yes'
# }

# #設定一個session
# rs = requests.session()

# #透過session記錄通過18禁頁面
# res1 = rs.post('https://www.ptt.cc/ask/over18',verify=False,data = playload)

# #進入文章開始爬取

# url = 'https://www.ptt.cc/bbs/Gossiping/M.1473225217.A.040.html'
# res = rs.get(url,verify=False)
# soup = BeautifulSoup(res.text)


# #date
# mydate = soup.select('.article-meta-value')[3].text

# # content
# content = soup.find(id="main-content").text
# target_content=u'※ 發信站: 批踢踢實業坊(ptt.cc),'
# content = content.split(target_content)
# content = content[0].split(mydate)
# main_content = content[1].replace('\n', ' ').replace('\t', ' ')
# print 'content:',main_content

# # message
# all_push_content = []
# num , good , bad , normal = 0,0,0,0

# for tag in soup.select('div.push'):
#     num += 1
#     push_tag = tag.find("span", {'class': 'push-tag'}).text
# #     print "push_tag:",push_tag
#     push_userid = tag.find("span", {'class': 'push-userid'}).text      
# #     print "push_userid:",push_userid
#     push_content = tag.find("span", {'class': 'push-content'}).text.replace('\n', '').replace('\t', '').strip()
#     push_content = push_content[1:].strip()
#     print "push_content:",push_content
#     push_ipdatetime = tag.find("span", {'class': 'push-ipdatetime'}).text.replace('\n', '').replace('\t', '').strip()
# #     print "push-ipdatetime:",push_ipdatetime
    
#     message = {
#         'push_tag':push_tag,
#         'push_userid':push_userid,
#         'push_content':push_content,
#         'push_datetime':push_ipdatetime,
#     }
    
#     all_push_content.append(message)

    
#     if push_tag == u'推 ':
#         good += 1
#     elif push_tag == u'噓 ':
#         bad += 1
#     else:
#         normal += 1
# push_count = {
#     'all':num,
#     'good':good,
#     'bad':bad,
#     'normal':normal,
#     'push_nums':good-bad,
# }

# print all_push_content
# print push_count

# print all_json_str[0]
# print all_json_str[0]['title']
# print all_json_str[0]['push_nums']
# print all_json_str[0]['main_content']
# print all_json_str[0]['url']
# print all_json_str[0]['push_content'][5]['content']
# print all_json_str[0]['push_count']['my_push_nums']
# print all_json_str[0]['push_count']['all']

## 神人寫好的爬蟲
- https://github.com/wy36101299/PTTcrawler